In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

# Logistic Regression

In [7]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(r'C:\Users\lksja\Downloads\archive\Base.csv')

In [8]:
df.tail()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
999995,0,0.8,0.124690,-1,143,30,0.051348,-0.826239,AB,530,6732.602414,3010.048099,3095.754245,42,8,CA,305,1,BB,1,1,31,0,1500.0,0,INTERNET,16.967770,other,0,1,0,7
999996,0,0.9,0.824544,-1,193,30,0.009591,0.008307,AC,408,1574.293294,2716.495767,4286.089050,0,5,CA,235,0,BA,1,1,-1,1,1000.0,0,INTERNET,1.504109,macintosh,0,1,0,7
999997,0,0.8,0.140891,-1,202,10,0.059287,50.609995,AA,749,1258.864938,3601.322892,3103.891664,2,3,CA,195,1,BE,0,1,31,0,200.0,0,INTERNET,16.068595,other,0,1,0,7
999998,0,0.9,0.002480,52,3,30,0.023357,-1.313387,AB,707,7048.137128,6521.395012,3068.265084,7,8,CA,148,0,BD,0,1,1,0,200.0,0,INTERNET,1.378683,linux,1,1,0,7
999999,0,0.6,0.993391,-1,174,30,0.020422,14.942456,AA,655,3737.076479,3135.788094,3051.003293,14,8,CA,100,1,BB,0,1,15,1,200.0,0,INTERNET,1.947926,other,1,1,0,7


In [9]:
# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns = ['payment_type', 'employment_status', 'device_os', 'housing_status'])

# Binary Encoding
df_encoded['source'] = df_encoded['source'].map({'INTERNET' : 1, 'TELEAPP' : 0})

# Normalization
def columns_to_normalize(df):

    list_of_columns = []
    
    for j in df.columns:
        if (pd.api.types.is_numeric_dtype(df[j].dtypes)) & (df[j].max() > 1 or df[j].min() < 0):
            list_of_columns.append(j)

    return list_of_columns
        

In [10]:
scaler = StandardScaler()
col_to_scale = columns_to_normalize(df_encoded)

df_encoded[col_to_scale] = scaler.fit_transform(df_encoded[col_to_scale])

In [11]:
df_encoded.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,payment_type_AA,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,employment_status_CA,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,device_os_linux,device_os_macintosh,device_os_other,device_os_windows,device_os_x11,housing_status_BA,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG
0,0,0.3,0.986506,-0.402272,-0.696644,0.524782,-0.189335,4.634885,-0.510946,2.469193,2.082983,2.050084,-0.390235,-0.894663,0.459380,1,0,1,-0.151797,0,2.01852,0,1,1.080517,1,-0.101305,0,-1.488092,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
1,0,0.8,0.617426,-0.402272,0.027285,-1.138310,-0.188711,-0.470003,0.084852,1.182299,0.659452,1.179919,-0.394586,1.687885,0.330221,1,1,1,-0.729504,0,2.01852,0,1,-0.520482,1,-0.101305,0,-1.488092,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False
2,0,0.8,0.996707,-0.175238,-0.821069,0.524782,-0.188298,-0.501671,-0.475139,-0.396701,0.474717,1.235244,-0.368478,0.297282,-0.602591,1,0,1,1.581324,0,-0.64782,0,1,1.890380,0,-0.101305,0,-1.488092,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False
3,0,0.6,0.475100,-0.129831,-0.821069,-0.306764,-0.189288,-0.520089,1.900097,2.913125,1.342311,1.211090,-0.377181,0.694597,-0.588240,1,0,1,-0.812033,0,-0.64782,0,1,0.954908,1,-0.101305,0,-1.488092,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
4,0,0.9,0.842307,-0.402272,-0.651398,0.524782,0.876453,1.902092,0.762212,0.643393,0.239496,1.178907,-0.398938,-0.696005,-0.573889,0,1,1,1.251206,0,-0.64782,0,1,-0.473278,0,-0.101305,0,-1.488092,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False


In [12]:
X = df_encoded.iloc[:, 1:].to_numpy()
y = df_encoded['fraud_bool'].to_numpy()

In [13]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

for train_idx, test_idx in split.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [15]:
b_in = 0
w_in = np.array(np.random.rand(51))

In [27]:
def compute_cost(x, y, w, b):

    m = x.shape[0]
    cost = 0
    epsilon = 1e-15
    gamma = 2
    
    z = x @ w + b # Iloczyn skalarny dla każdej z obserwacji
    f_wb = 1 / (1 + np.exp(-z.astype('float64')))
    focal_loss = -y * ((1 - f_wb)**gamma) * np.log(f_wb + epsilon) - (1 - y) * ((f_wb)**gamma) * np.log(1 - f_wb + epsilon)
    cost = np.sum(focal_loss) / m
    
    return cost

In [28]:
compute_cost(X_train, y_train, w_in, b_in)

np.float64(4.123704194032476)

In [18]:
def compute_gradient(x, y, w, b):

    m = x.shape[0]

    z = x @ w + b
    f_wb = 1/(1 + np.exp(-z.astype('float64')))
    error = f_wb - y

    dj_dw = 1/m * np.dot(x.T, error)
    dj_db = 1/m * np.sum(error)

    return dj_dw, dj_db

In [42]:
def compute_gradient_focal(x, y, w, b):

    gamma = 2
    epsilon = 1e-15
    m = x.shape[0]

    z = x @ w + b
    p = 1/(1 + np.exp(-z.astype('float64')))
    error = p - y

    dj_dp = ((y * (1 - p)**(gamma - 1))*(gamma * np.log(p + epsilon) - ((1 - p) / p)) \
             -((1 - y)*p**(gamma - 1)) * (gamma * np.log(1 - p + epsilon) - (p / (1 - p + epsilon))))

    dp_dz = p * (1 - p)
    dj_dz = dj_dp * dp_dz

    dj_dw = (1 / m) * np.dot(x.T, dj_dz)
    dj_db = (1 / m) * np.sum(dj_dz)
    
    return dj_dw, dj_db
    

In [43]:
compute_gradient_focal(X_train, y_train, w_in, b_in)

(array([0.5644788410254318, 0.49327709827468996, 0.0073858277024989125,
        0.007708820804475003, 0.01986219886071862, 0.0028381393289172324,
        0.015517414415729043, 0.01606037277156673, 0.028922195790724678,
        0.03052948518007618, 0.03429248139495258, 0.009226729395266376,
        0.014354020126687498, 0.01558567544081887, 0.5363859102975411,
        0.41937348982167905, 0.8903516124071202, 0.02739748256900859,
        0.22808460723538468, 0.00884511460492297, 0.025436713277383716,
        0.9903515889650432, 0.009613433402457048, 0.5769861297038583,
        0.0012553137875182738, 0.0, -0.02868951296751378,
        0.2660160521113429, 0.37379896781272076, 0.23737357910679438,
        0.11913487645514399, 0.00029389477007257743, 0.7307353747284273,
        0.13907136978712467, 0.03773985371137785, 0.02466639539272754,
        0.021748688342732178, 0.04220469074464291, 0.00045099754901477756,
        0.3309093399116426, 0.05317181915412688, 0.33963802073361204,
        0

In [48]:
def gradient_descent(x, y, w, b, gradient_function, cost_function, alpha, num_iter):

    cost_history = []

    for j in range(num_iter):
        dj_dw, dj_db = gradient_function(x, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        if j % 50 == 0:
            cost = cost_function(x, y, w, b)
            cost_history.append(cost)
            print(f"Koszt iteracji nr.:{j} wyniósł: {cost}")
            
    return w, b, cost_history

In [49]:
num_iter = 801
alpha = 0.01

In [47]:
%%time
w_final, b_final, cost_history = gradient_descent(X_train, y_train, w_in, b_in, compute_gradient_focal, compute_cost, alpha, num_iter)

Koszt iteracji nr.:0 wyniósł: 4.067900285751895
Koszt iteracji nr.:50 wyniósł: 1.8229861155941642
CPU times: total: 5min 46s
Wall time: 5min 51s


# Recall, Precision, F1-Score

In [387]:
faktyczny_fraud = np.flatnonzero(y_test) # Indeksy, które są fraud
moje_typy_na_true = np.flatnonzero(predictions_values) # Indeksy, które JA wskazałem, że są fraud
mask = np.isin(moje_typy_na_true, faktyczny_fraud) # Sprawdzam, które z indeksów wytypowałem dobrze

brak_fraud = np.argwhere(y_test == 0).ravel() # Transakcje, które nie były fraudem
moje_typy_na_false = np.argwhere(predictions_values == 0).ravel() # Transakcje, które ja wytypowałem na 0, brak fraudu
mask2 = np.isin(moje_typy_na_false, brak_fraud) # Mask, kiedy miałem rację, że dana transakcja to fraud

In [395]:
true_positive = prediction[mask] # Ile udało się przewidzieć ###########################

false_positive = prediction[~mask] # Liczba false-flag

true_negative = moje_typy_na_false[mask2] ############################

false_negative = moje_typy_na_false[~mask2]

In [397]:
precision = len(true_positive) / (len(true_positive) + len(false_positive))

In [399]:
recall = len(true_positive) / (len(true_positive) + len(false_negative))

In [401]:
f1_score = 2 * precision * recall/(precision + recall)